In [1]:
import torch

In [2]:
bruh = torch.nn.Linear(1, 1)

In [3]:
bruh.parameters()

<generator object Module.parameters at 0x10714ac70>

In [2]:
multihead = torch.nn.MultiheadAttention(512, 2)

In [3]:
query = key = value = torch.rand(4, 512)

In [3]:
mask = torch.tril(torch.ones(4, 4, 4))

In [6]:
a,b,c = mask.size()

In [7]:
a,b,c

(4, 4, 4)

In [14]:
test = torch.arange(0, torch.prod(torch.tensor(mask.shape))).reshape_as(mask)

In [17]:
test[:,:, 0::2]

tensor([[[ 0,  2],
         [ 4,  6],
         [ 8, 10],
         [12, 14]],

        [[16, 18],
         [20, 22],
         [24, 26],
         [28, 30]],

        [[32, 34],
         [36, 38],
         [40, 42],
         [44, 46]],

        [[48, 50],
         [52, 54],
         [56, 58],
         [60, 62]]])

In [18]:
test[:,:, 1::2]

tensor([[[ 1,  3],
         [ 5,  7],
         [ 9, 11],
         [13, 15]],

        [[17, 19],
         [21, 23],
         [25, 27],
         [29, 31]],

        [[33, 35],
         [37, 39],
         [41, 43],
         [45, 47]],

        [[49, 51],
         [53, 55],
         [57, 59],
         [61, 63]]])

In [75]:
def positional_encoding(x: torch.Tensor, embed_weights: torch.Tensor):
    pos = torch.arange(0, x.size(1), dtype=torch.float)
    pos_encoding = torch.stack(
        [pos / pow(10000, (2 * dim // 2) / x.size(-1)) for dim in range(x.size(-1))],
    ).transpose_(0, 1)
    print(pos_encoding.shape)
    # assign positional encodings as non-trainable weights
    embed_weights.requires_grad = False
    embed_weights[:, 0::2] = torch.sin(pos_encoding[:, 0::2])
    embed_weights[:, 1::2] = torch.cos(pos_encoding[:, 1::2])

In [81]:
embed = torch.nn.Embedding(4, 512)
wei = embed.weight
x = torch.rand(4, 4, 512)

In [85]:
out = embed(torch.randint(0, 4, (4, 4)))

In [86]:
out.shape

torch.Size([4, 4, 512])

In [80]:
wei.shape

torch.Size([4, 512])

In [77]:
positional_encoding(x, wei)

torch.Size([4, 512])


In [79]:
wei.shape

torch.Size([4, 512])

In [5]:
attn_out, attn_weights = multihead(
    query,
    key,
    value,
    need_weights=True,
    attn_mask=mask,
    is_causal=True,
    average_attn_weights=False,
)

In [6]:
attn_weights

tensor([[[0.4405, 0.1944, 0.1805, 0.1846],
         [0.3297, 0.4034, 0.1335, 0.1334],
         [0.2606, 0.3206, 0.2937, 0.1251],
         [0.2115, 0.2641, 0.2601, 0.2643]],

        [[0.4797, 0.1760, 0.1547, 0.1896],
         [0.3552, 0.3697, 0.1316, 0.1434],
         [0.3056, 0.2928, 0.2864, 0.1152],
         [0.2538, 0.2417, 0.2456, 0.2589]]], grad_fn=<SqueezeBackward1>)

In [7]:
attn_out

tensor([[-0.1563, -0.4218, -0.1676,  ...,  0.0779, -0.1374, -0.0954],
        [-0.1182, -0.4663, -0.1266,  ...,  0.0345, -0.1591, -0.1155],
        [-0.1184, -0.4627, -0.1247,  ...,  0.0680, -0.1185, -0.1001],
        [-0.0914, -0.4527, -0.1364,  ...,  0.0766, -0.1253, -0.0670]],
       grad_fn=<SqueezeBackward1>)